### Initialize Declarativewidgets

In [ ]:
library("declarativewidgets")

In [ ]:
initWidgets()

Import Polymer elements:

In [ ]:
IRdisplay::display_html("
<link rel='import' href='urth_components/paper-slider/paper-slider.html'
        is='urth-core-import' package='PolymerElements/paper-slider'>
<link rel='import' href='urth_components/urth-viz-table/urth-viz-table.html'
        is='urth-core-import'>
<link rel='stylesheet' type='text/css' href='bcard.css'>
<link rel='import' href='urth_components/urth-viz-pie/urth-viz-pie.html' 
        is='urth-core-import'>
<link rel='import' href='urth_components/urth-viz-table/urth-viz-table.html' 
        is='urth-core-import'>
<link rel='import' href='urth_components/urth-viz-bar/urth-viz-bar.html' 
        is='urth-core-import'>
<link rel='import' href='urth_components/urth-viz-explorer/urth-viz-explorer.html' 
        is='urth-core-import'>
<link rel='import' href='urth_components/paper-input/paper-input.html' 
        is='urth-core-import' package='PolymerElements/paper-input'>
<link rel='import' href='urth_components/paper-spinner/paper-spinner.html' 
        is='urth-core-import' package='PolymerElements/paper-spinner'>
")

### Function widget

In [ ]:
math = function(x, y) {return (x*y)}

In [ ]:
IRdisplay::display_html("
<template is='dom-bind'>
<urth-core-function ref='math' arg-x='{{x}}' arg-y='{{y}}' result='{{res}}' auto></urth-core-function>
    <label>x:</label><paper-slider min='10' max='100' step='1' value='{{x}}'></paper-slider><span>{{x}}</span><br>
    <label>y:</label><paper-slider min='1' max='100' step='1' value='{{y}}'></paper-slider><span>{{y}}</span><br>
Result: <span>{{res}}</span>
</template>
")

### Channels

#### Setting data in a channel

In [ ]:
IRdisplay::display_html("
<template is='urth-core-bind' channel='a'>
    <div>Hello from <span>{{user}}</span></div>
</template>
")

In [ ]:
channel("a")$set("user", "mike p")

#### Watching data in a channel

In [ ]:
IRdisplay::display_html("
<template is='urth-core-bind' channel='a'>
    Name: <input value='{{user::input}}'></input>
    <p>
        <span>{{msg}}</span>
    </p>
</template>
")

In [ ]:
handler <- function(old_val, new_val) {
    print("in handler")
    print(old_val)
    print(new_val)
    channel("a")$set("msg", paste("Watch triggers for new value {",new_val,"}, and old value {",old_val,"}."))
}

In [ ]:
channel("a")$watch("user", handler)

### DataFrame widget

In [ ]:
First_Name <- c("John", "Jane", "Joe")
Last_Name <- c("Johnson", "Doe", "Smith")
Role <- c("Web Developer", "Software Engineer", "Rockstar Dev")
Amount <- c("500", "456", "4526")
Bigger_Number <- c("1234325431", "1434215411", "1237328421")
Website <- c("http://javi.er", "http://www.ibm.us", "http://cooldevs.org/xavier")
aDataFrame <- data.frame(First_Name, Last_Name, Role, Amount, Bigger_Number, Website)

In [ ]:
IRdisplay::display_html("
<template id='foo' is='dom-bind'>
    <urth-core-dataframe ref='aDataFrame' limit='10' value='{{x}}'></urth-core-dataframe>
    <urth-viz-pie primary='0' secondary='3'datarows='{{ x.data }}' columns='{{ x.columns }}'></urth-viz-pie>

    <urth-viz-table datarows='{{ x.data }}' selection='{{sel}}' columns='{{ x.columns }}' selection-as-object>
        <urth-viz-col index='4' format='$0,0.00' type='numeric'></urth-viz-col>
        <urth-viz-col index='3' format='$0,0.0' type='numeric'></urth-viz-col>
    </urth-viz-table>

    <div class='bcard'>
        <div class='info'>
            <div class='line full-name'><span>{{sel.First_Name}}</span> <span>{{sel.Last_Name}}</span></div>
            <span class='line title'>{{sel.Role}}</span>
            <span class='line phone-number'>{{sel.Amount}}</span>
            <span class='line email'>{{sel.Bigger_Number}}</span>
            <span class='line website'>{{sel.Website}}</span>
        </div>
        <div class='logo'></div>
    </div>
</template>
")

### Spark DataFrame widget

In [ ]:
library(SparkR)
sc <- sparkR.init()
sqlContext <- sparkRSQL.init(sc)

In [ ]:
sparkDataFrame <- createDataFrame(sqlContext, aDataFrame)

In [ ]:
IRdisplay::display_html("
<template id='foo' is='dom-bind'>
    <urth-core-dataframe ref='sparkDataFrame' limit='10' value='{{x}}'></urth-core-dataframe>
    <urth-viz-pie primary='0' secondary='3'datarows='{{ x.data }}' columns='{{ x.columns }}'></urth-viz-pie>

    <urth-viz-table datarows='{{ x.data }}' selection='{{sel}}' columns='{{ x.columns }}' selection-as-object>
        <urth-viz-col index='4' format='$0,0.00' type='numeric'></urth-viz-col>
        <urth-viz-col index='3' format='$0,0.0' type='numeric'></urth-viz-col>
    </urth-viz-table>

    <div class='bcard'>
        <div class='info'>
            <div class='line full-name'><span>{{sel.First_Name}}</span> <span>{{sel.Last_Name}}</span></div>
            <span class='line title'>{{sel.Role}}</span>
            <span class='line phone-number'>{{sel.Amount}}</span>
            <span class='line email'>{{sel.Bigger_Number}}</span>
            <span class='line website'>{{sel.Website}}</span>
        </div>
        <div class='logo'></div>
    </div>
</template>
")

### Function returning a custom class. Defining a custom serializer
In this example, we have a function that returns a custom type Foo. We would like to serialize the Foo that's returned so that the value is useful for other widgets.

In [ ]:
library(R6)
Foo <- R6Class(
    'Foo',
    public = list(
        foo = function() {
            return ("The foo is strong with this one!")
        }
    )
)

Now define the custom serializer for Foo by implementing the BaseSerializer methods.
    
For our serialize method, we'll call obj$foo()

In [ ]:
Foo_Serializer <- R6Class(
    'Foo_Serializer',
    inherit = Serializer,
    public = list(
        klass = function() {
            return ("Foo")
        },
        serialize = function(obj) {
            return (obj$foo())
        },
        check_packages = function() {
            tryCatch({
                library(base)
            }, error = function(e) {
                return (False)
            })
            return (True)
        },
        initialize = function() {
            #initialize must exists on this class
        }
    )
)

register_serializer(Foo_Serializer$new())

In [ ]:
aFooFunction <- function() {
    return (Foo$new())
}

Now when we invoke, the function will return the Foo instance, and we'll see the serialized return value:

In [ ]:
IRdisplay::display_html("
    <template is='dom-bind'>
        <urth-core-function id='f3f' ref='aFooFunction' result='{{r}}'></urth-core-function>
        <button onClick='f3f.invoke()'>invoke</button>    
        <span>{{r}}</span>
    </template>
")